In [5]:
%matplotlib widget

import io
import sys
import os
import os.path as op
from glob import glob

import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import mne
from mne.io import read_epochs_eeglab
from mne.decoding import SSD
from mne.time_frequency import psd_array_multitaper

In [6]:
# set data directory
data_dir = '/gpfs/data/brainstorm-ws/data/'
data_type = 'TRAINING'

In [7]:
# read metadata from master list
fname_master = op.join(data_dir, data_type, data_type + '_Master List_All Series.csv')
master_keys = np.loadtxt(fname_master,  delimiter=',', max_rows=1, dtype=str)
master_keys[0] = 'TMSID'  # fix first key string
master_values = np.loadtxt(fname_master, delimiter=',', skiprows=1, dtype=str)
master_info = {key: master_values[:, master_keys == key].squeeze() for key in master_keys}

# get subject IDs for datasets with both pre-/post-TMS EEG recordings
subj_select = master_info['HasT1T2'] == '1'
subj_ids = master_info['TMSID'][subj_select]

In [19]:
# read metadata from demographics and clinical outcomes
fname_outcomes = op.join(data_dir, data_type, data_type + '_Demographics and Clinical Outcomes_All Series.csv')
outcomes_keys = np.loadtxt(fname_outcomes,  delimiter=',', max_rows=1, dtype=str)
outcomes_keys[0] = 'TMSID'  # fix first key string
outcomes_values = np.loadtxt(fname_outcomes, delimiter=',', skiprows=1, dtype=str)
outcome_info = {key: outcomes_values[:, outcomes_keys == key].squeeze() for key in outcomes_keys}

# get demographic and outcome data
subj_select = np.in1d(outcome_info['TMSID'], subj_ids)
subj_age = outcome_info['AgeTMSstart'][subj_select].astype(int)
subj_sex = outcome_info['Sex'][subj_select].astype(int)
subj_HSD = outcome_info['SevHxDep'][subj_select].astype(int)
subj_response = outcome_info['LastRespIDSSR'][subj_select].astype(int)

# get series
subj_series = outcome_info['Series'][subj_select].astype(int)

In [ ]:
data_dir = '/gpfs/data/brainstorm-ws/data/TRAINING/series_1/'
subj_id = '0372'
t1_fname = op.join(data_dir, 'T1/preprocessed', subj_id + '_Pre_Session1_Series1_Preprocessed.set')
t2_fname = op.join(data_dir, 'T2/preprocessed', subj_id + '_Post_Session1_Series1_Preprocessed.set')

epochs_t1 = read_epochs_eeglab(t1_fname)
epochs_t2 = read_epochs_eeglab(t2_fname)

epochs_t1 = epochs_t1.pick_channels(['C3'])
epochs_t2 = epochs_t2.pick_channels(['C3'])

In [ ]:
plt.close('all')
fig, ax = plt.subplots(1, 1)

epochs_t1.plot_psd(fmin=3., fmax=50., ax=ax, color='orange', spatial_colors=False)
epochs_t2.plot_psd(fmin=3., fmax=50., ax=ax, color='blue', spatial_colors=False)
    
plt.show()

In [ ]:
data_t1 = epochs_t1.get_data().squeeze()  # epoch x time
data_t2 = epochs_t2.get_data().squeeze()  # epoch x time

times = epochs_t1.times
dt = times[1] - times[0]
samp_freq = 1 / dt
print(np.array_equal(epochs_t1.times, epochs_t2.times))

In [ ]:
import os

data_dir_t1 = op.join(data_dir, 'T1/preprocessed')

fnames = sorted(os.listdir(data_dir_t1))